In [0]:
%sql
SELECT * FROM parquet. `/Volumes/workspace/default/yellow-taxi`;

In [0]:
%sql
SELECT * FROM read_files(
'/Volumes/workspace/default/yellow-taxi',
format => 'parquet'
) ;

In [0]:
%sql
DROP TABLE IF EXISTS default.yellow_taxi_bronze;
CREATE TABLE default.yellow_taxi_bronze AS
SELECT
  *,
  cast(tpep_pickup_datetime AS DATE) AS pickup_datetime,
  cast(tpep_dropoff_datetime AS DATE) AS dropoff_datetime,
  _metadata.file_modification_time AS file_modification_time,
  _metadata.file_name AS source_file,
  current_timestamp() as ingestion_time
FROM read_files(
  '/Volumes/workspace/default/yellow-taxi',
  format => 'parquet')
;



In [0]:
%sql
SHOW COLUMNS FROM default.yellow_taxi_bronze

In [0]:
%sql
DESCRIBE TABLE EXTENDED workspace.default.yellow_taxi_bronze;

In [0]:
%sql

COPY INTO default.yellow_taxi_bronze
FROM '/Volumes/workspace/default/yellow-taxi'
FILEFORMAT = PARQUET VALIDATE
COPY_OPTIONS('mergeSchema' = 'true');

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE default.yellow_taxi_silver AS
SELECT
    VendorID,
    CAST(tpep_pickup_datetime AS DATE) AS pickup_date,
    CAST(tpep_dropoff_datetime AS DATE) AS dropoff_date,
    passenger_count,
    trip_distance,
    RatecodeID,
    store_and_fwd_flag,
    PULocationID,
    DOLocationID,
    payment_type,
    fare_amount,
    extra,
    mta_tax,
    tip_amount,
    tolls_amount,
    improvement_surcharge,
    total_amount,
    congestion_surcharge,
    CAST(processing_time AS TIMESTAMP) AS processing_time,
    source_file
FROM default.yellow_taxi_bronze
""")

In [0]:
display(spark.sql("SELECT * FROM default.yellow_taxi_silver"))

In [0]:
%sql

CREATE OR REPLACE VIEW default.yellow_taxi_by_pickup
AS
SELECT
    date(pickup_datetime) AS pickup_time,
    count(*) AS total_daily_pickups
FROM default.yellow_taxi_silver
GROUP BY date(pickup_datetime);

In [0]:
display(spark.sql("SELECT * FROM default.yellow_taxi_by_pickup"))